# Imports

##### General imports

In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm

##### Import from flatland environment 

In [4]:
from flatland.envs.rail_env import RailEnv
from flatland.envs.observations import *
from flatland.envs.rail_generators import complex_rail_generator,rail_from_manual_specifications_generator,random_rail_generator, RailGenerator,sparse_rail_generator
from flatland.envs.schedule_generators import complex_schedule_generator, random_schedule_generator, ScheduleGenerator, sparse_schedule_generator
from flatland.utils.rendertools import RenderTool, AgentRenderVariant

##### Import from our framework

In [5]:
from src.graph import NetworkGraph

In [44]:
from src.flows import *

# Test of time expanded network

##### Create a flatland network

sparse netwrok

In [8]:
number_agents = 5

size_side = 50
stochastic_data = {'prop_malfunction': 0.3,  # Percentage of defective agents
                   'malfunction_rate': 30,  # Rate of malfunction occurence
                   'min_duration': 3,  # Minimal duration of malfunction
                   'max_duration': 20  # Max duration of malfunction
                   }
speed_ration_map = {1.: 0.25,  # Fast passenger train
                    1. / 2.: 0.25,  # Fast freight train
                    1. / 3.: 0.25,  # Slow commuter train
                    1. / 4.: 0.25}  # Slow freight train
env = RailEnv(width=size_side,
              height=size_side,
              rail_generator=sparse_rail_generator(max_num_cities=4,
                                                   # Number of cities in map (where train stations are)
                                                   seed=14,  # Random seed
                                                   grid_mode=False,
                                                   max_rails_between_cities=2,
                                                   max_rails_in_city=8,
                                                   ),
              schedule_generator=sparse_schedule_generator(speed_ration_map),
              number_of_agents=number_agents,
              stochastic_data=stochastic_data,  # Malfunction data generator
              obs_builder_object=GlobalObsForRailEnv(),
              remove_agents_at_target=True
              )

# RailEnv.DEPOT_POSITION = lambda agent, agent_handle : (agent_handle % env.height,0)
env.reset()

env_renderer = RenderTool(env,
                          agent_render_variant=AgentRenderVariant.AGENT_SHOWS_OPTIONS_AND_BOX,
                          show_debug=True,
                          screen_height=1100,
                          screen_width=1800)
env_renderer.reset()

env_renderer.render_env(show=True, show_observations=False, show_predictions=False)



SystemExit: [ABORT] Cannot fit more than one city in this map, no feasible environment possible! Aborting.

/home/charles/anaconda3/envs/flatland-rl/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


usual network

In [9]:
number_agents = 10

size_side = 20
env = RailEnv(width=size_side,
              height=size_side,
              rail_generator=complex_rail_generator(nr_start_goal=number_agents, nr_extra=1, 
                                                    min_dist=6, max_dist=99999, 
                                                    seed = np.random.randint(0,2000)),
              schedule_generator=complex_schedule_generator(),
              number_of_agents=number_agents,
              obs_builder_object=GlobalObsForRailEnv())

env.reset()



env_renderer = RenderTool(env)
env_renderer.render_env(show=True, show_predictions=False, show_observations=False)

In [10]:
matrix_rail = np.array(env.rail.grid.tolist())
flatlandNetwork = NetworkGraph(matrix_rail,[(0,1)],[(1,0)])

In [11]:
sources = []
sinks = []
directions = []
for agent in env.agents:
    sources.append(agent.initial_position)
    sinks.append(agent.target)
    try:
        directions.append(agent.direction)
    except:
        pass

In [12]:
if len(directions) == 0:
    directions = None

##### create a time expanded network

In [13]:
import time

In [14]:
start = time.time()
TestNetworkTime = TimeNetwork(flatlandNetwork, depth=200)
stop = time.time()
print(f'time taken to build the graph: {stop-start}')

time taken to build the graph: 2.6717913150787354


In [15]:
TestNetworkTime.connect_sources_and_sink(sources,sinks,directions = None)

### create initial solution

In [19]:
from tqdm import tqdm

In [16]:
constraints, find_constraints = TestNetworkTime.get_topology_network()

In [23]:
for i,(j,k) in enumerate(tqdm(zip(sources,sinks))):
    print(j,k)

10it [00:00, 3821.69it/s]

(14, 2) (7, 14)
(17, 0) (15, 17)
(3, 5) (8, 9)
(2, 10) (0, 4)
(0, 6) (17, 19)
(1, 14) (16, 2)
(14, 0) (6, 3)
(19, 10) (14, 14)
(12, 2) (3, 0)
(11, 9) (2, 19)


In [200]:
test = InitialSolutionGenerator(TestNetworkTime,constraints,find_constraints,len(sources))

In [201]:
solution = test.getInitialSolution()

10it [00:01,  6.60it/s]


In [202]:
test.showStats()

{'not shortest path ': 6}


In [240]:
from src.flows.PricingProblem import PricingSolver

In [ ]:
flag = True
iteration = 0
master = MasterProblem.MasterProblem(solution,constraints,find_constraints,number_agents)
master.build()
pricingSolver = PricingSolver(TestNetworkTime.graph,constraints,find_constraints,len(sources))
while flag:
    print("iteration ",iteration)
    iteration+= 1
    master.solveRelaxedModel()
    duals = master.getDualVariables()
    pathsToAdd, flag = pricingSolver.get_columns_to_add(master.getDualVariables(),master.constraintsActivated)
    master.addColumn(pathsToAdd)

iteration  0
Optimize a model with 492 rows, 10 columns and 492 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 10 rows and 502 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  3.400000000e+02
iteration  1
Optimize a model with 658 rows, 12 columns and 663 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 12 rows and 670 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.4000000e+02   0.000000e+00   0.000000e+00      0s


  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 7 rows and 658 columns
Presolve time: 0.02s
Presolved: 44 rows, 51 columns, 172 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7900000e+02   0.000000e+00   3.000000e+00      0s
       5    3.4000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.03 seconds
Optimal objective  3.400000000e+02
iteration  15
Optimize a model with 658 rows, 54 columns and 3855 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 7 rows and 658 columns
Presolve time: 0.01s
Presolved: 47 rows, 54 columns, 184 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.7900000e+02   0.000000e+00   3.000000e+00      0s
       5    3.40000

In [244]:
master.getDualVariables()

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -4.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.

In [245]:
len(master.constraintsActivated)

648

##### Test LP Formulation

test a simple graph

In [ ]:
import datetime as dt
print(dt.datetime.now())

In [ ]:
constraints,findConstraints = TestNetworkTime.get_topology_network()

In [ ]:
mcflow = MCFlow(TestNetworkTime.graph,len(sources),constraints,integer = False)

In [ ]:
mcflow.solve()

In [ ]:
paths = mcflow.get_paths_solution()

In [ ]:
score = 0
for _,path in paths.items():
    score += int(len(path)/2)
    
print(f"score {score}")

In [ ]:
lengths = []
pathsToAllongate = copy.deepcopy(mcflow.solution)
for agent,path in pathsToAllongate.items():
    lengths.append(len(path))
    
maxLength = max(lengths)
for agent,path in pathsToAllongate.items():
    for i in range(maxLength-len(path)):
        path.append(None)
    pathsToAllongate[agent] = path

In [ ]:
dfPaths = pd.DataFrame(pathsToAllongate)
dfPaths

In [ ]:
import datetime as dt
print(dt.datetime.now())

In [ ]:
colors = [(230, 25, 75), (60, 180, 75), (255, 225, 25), (0, 130, 200), (245, 130, 48),
          (145, 30, 180), (70, 240, 240), (240, 50, 230), (210, 245, 60), (250, 190, 190),
          (0, 128, 128), (230, 190, 255), (170, 110, 40), (255, 250, 200), (128, 0, 0), (170, 255, 195),
          (128, 128, 0), (255, 215, 180), (0, 0, 128), (128, 128, 128), (255, 255, 255), (0, 0, 0)]
colors = [(x[0]/255.,x[1]/255.,x[2]/255.) for x in colors]

In [ ]:
flatlandNetwork.show(paths = solutions)